<center><font color=#555555><font size=6> MAF1731 - Microstructure and Trading Systems </font> <br> <br>
    
<center><font color=#555555><font size=5> B.Eng Financial Engineering </font> <br> <br> <br> <br>
   
<center> <font color= #555555> <font size = 4> May 2023 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/'>Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Module 1 : Part 1 </font> <br> <br> <font color= #555555> <font size = 5> Market Microstructure of Blockchain-Based Financial Assets (OrderBooks) </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

# <font color= #6B6B6B> <font size = 6> 1. Introduction </font>
<hr style="border:0.02in solid gray"> </hr>

This notebook is provided as a case of study of Blockchain-Based financial assets, in particular, Bitcoin and Ethereum cryptocurrencies. Data within this notebook was extracted from the Kraken exchange and making use of the ccxt library. 

<br>

# <font color= #6B6B6B> 2. Install/Load Packages and Depedencies </font>
<hr style="border:0.02in solid gray"> </hr>

<br>

## <font color= #6B6B6B> 2.2 Python Packages </font> <font color= #555555>

In order to run this notebook, it is necessary to have installed and/or have the _requirements.txt_ file with the following: 

- pandas==1.3.3 
- numpy==1.21.2
- jupyter==1.0.0
- ccxt==1.59.95

## <font color= #6B6B6B> 2.3 Files Dependencies </font> <font color= #555555>

The following are the file dependencies that are needed to run this notebook:

- Files/orderbooks_05jul21.json : JSON file with Historical Orderbooks Gathered from BitFinex and Kraken for BTC/USDT

## <font color= #6E6E6E> 2.3 Install Packages</font>

In [1]:
%%capture

# Install all the pip packages in the requirements.txt
import sys
!{sys.executable} -m pip install -r requirements.txt

## <font color= #6E6E6E> 2.4 Load Packages</font>

In [2]:
# -- Generic
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pyarrow.parquet as pq

# -- Visualization
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'pyarrow'

## <font color= #6E6E6E> 2.5 Load Other Scripts </font>

In [ ]:
# -- Project scripts
import visualizations as vs
import functions as fn
import data as dt

# <font color= #6B6B6B> 3. Read and Visualize Data </font>
<hr style="border:0.02in solid gray"> </hr>

Historical Orderbook data is stored in a **JSON** formated file, which stands for **J**ava **S**cript **O**bject **N**otation, in practical terms, it is very similar to a python dictionary since it has a list-like structure of storing the data, with one or several keys and each of which can have more keys and or data points.

Remember also that a Pandas DataFrame is also in its core form a dictionary where each colum is a key and the column contents are each key contents. So, in a way, a **JSON** file is one option to represent a multidimensional DataFrame. 

## <font color= #6B6B6B> 3.1 Load and display the data </font>

To read the contents of the file is very simple, by using the library *json*, it is a pre-installed one and you do need to import it but its already installed in the python base installation. 

In [ ]:
orderbooks_data = dt.read_file(file_name='orderbooks_05jul21.json', 
                               folder_route= 'files/')

Lets check the contents of the file.

In [ ]:
orderbooks_data.keys()

Allright, we have 2 keys, each of which corresponds to different exchanges where *BTC/USDT* is listed. Lets get store the timestamps of the two in sepparate objects, we will need those for later calculations.

In [ ]:
bitfinex_ts = list(orderbooks_data['bitfinex'].keys())
kraken_ts = list(orderbooks_data['kraken'].keys())

Now lets see some other info of each of the list, in priciple we are interested in just one for now so this calculation can apply to any of those two. Lets use as an example the data we have from Bitfinex.

## <font color= #6B6B6B> Q1 : Can you create a function for timestamps comparisson ? (2pt) </font>
<hr style="border:0.02in solid gray"> </hr>

Inside the functions file, create a function named *f_timestamps_info* with input parameteres *ts_list_o* and *ts_list_d*, and the output must be a *dict* type object with the following keys and content:

- *'first_o'* : has the first timestamp of the origin exchange.
- *'last_o'* : has the last timestamp of the origin exchange.
- *'qty_o'* : has the total amount of timestamps of the origin exchange.
- *'first_d'* : has the first timestamp of the destination exchange.
- *'last_d'* : has the last timestamp of the destination exchange.
- *'qty_d'* : has the total amount of timestamps of the destination exchange.
- *'exact_match'* : {'qty': The quantity of timestamps that were **exactly** the same in both origin and destination. *(Leave a 0 if none where present)*, 'values': a list str values of the timestamps that are **exactly** the same in both origin and destination, *(Leave an empty list [] if none where present)*}

Place the code for this function in the *functions.py* file and should be callable from the following cell.

In [ ]:
def f_timestamps_info(ts_list_o, ts_list_d):
    result = {}
    
    # Origin exchange
    result['first_o'] = ts_list_o[0] if ts_list_o else None
    result['last_o'] = ts_list_o[-1] if ts_list_o else None
    result['qty_o'] = len(ts_list_o)
    
    # Destination exchange
    result['first_d'] = ts_list_d[0] if ts_list_d else None
    result['last_d'] = ts_list_d[-1] if ts_list_d else None
    result['qty_d'] = len(ts_list_d)
    
    # Exact match
    exact_match = set(ts_list_o) & set(ts_list_d)
    result['exact_match'] = {
        'qty': len(exact_match),
        'values': list(exact_match)
    }
    
    return result

To practice the use of *display* function on jupyter notebooks, you should delivery the exact same result

In [ ]:
# Delete this comment and place your code here in this cell

In [ ]:
display(bitfinex_ts[0:4])
display(kraken_ts[0:4])

From now and on for the purpose of this lab, we are going to use only the information of one exchange, and also we are going to have the following order of columns (it is just a convention) 'bid_size', 'bid', 'ask' and 'ask_size'

In [ ]:
ob_data = orderbooks_data['kraken']

## <font color= #6B6B6B> Q2 : Can you get only the keys with not *None* values ? (2pt) </font>
<hr style="border:0.02in solid gray"> </hr>

There exists a posibility of having *None* or empty values for any timestamp contained in the data set, so for the next sections on this lab we need to use the keys of the dict with *not None* values.

Next incllude your code that generates the output that is showed.

In [ ]:
display('Number of historical Kraken orderbooks:')
display(f'Before dorpping Nones: {len(ob_data_no+ob_data_yes)}')
display(f'After dropping Nones: {len(ob_data)}')# Delete this comment and place your code here in this cell

Lets see the first timestamp from where we do have data

In [ ]:
display('First timestamp: ' + list(ob_data.keys())[0])
print('')
display('The Orderbook data is the following:')
print('')
display(pd.DataFrame(ob_data[list(ob_data.keys())[0]]))

... And what about the last timestamp we have data from?

In [ ]:
display('Last timestamp: ' + list(ob_data.keys())[-1])
print('')
display('The Orderbook data is the following:')
print('')
display(pd.DataFrame(ob_data[list(ob_data.keys())[-1]]))

All right, looks good. We just need to switch the columns in order as its more often used in industry. After that, we should make a plot to have a different visual representation of that data, lets use *plotly*

In [ ]:
ob_data = {orderbook: pd.DataFrame(ob_data[orderbook])[['bid_size', 'bid', 'ask', 'ask_size']]
           for orderbook in list(ob_data.keys())}

In [ ]:
ob_data[list(ob_data.keys())[0]]

## <font color= #6B6B6B> Q3 : Make a horizontal barplot for the Orderbook representation (4pt) </font>
<hr style="border:0.02in solid gray"> </hr>

Inside the visualizations.py file, there is a function named *plot_orderbook* with input parameteres *ob_data_plot*, using that function, you need to first select the day that had the highest total volume in the orderbook, and then use it to plot the first 20 levels for Bid side and first 20 levels for Ask side.

Note: If you find more than 1 timestamp with the exact same volume, use the one that is closest to the present.

In [ ]:
ob_data_plot=fn.data_plot(ob=ob_data)

max_vol_ts=ob_data_plot[1]
ob_data_plot=ob_data_plot[0]

display(ob_data_plot)

In [ ]:
ob_data_plot.head()

In [ ]:
plot_ob = vs.plot_ob(ob_data_plot=ob_data_plot, 
                     main_title='Orderbook Timestamp: ' + ob_keys[max_vol_ts])
plot_ob.show()

## <font color= #6B6B6B> Q4 : Can you calculate the historical bid, ask and spread ? (2pt) </font>
<hr style="border:0.02in solid gray"> </hr>

Now we need to calculate the historical bid, ask and spread. That means to calculate them using the Top of the Book, i.e. using only the best bid, the bes ask. So, it turns out we need to iterate over all orderbook data we have and calculate those values.

So, create a DataFrame, *df_ts_tob* with the following contents:

In [ ]:
def calculate_tob_values(ob_data):
    tob_data = []
    for ob in ob_data:
        date = ob['date']
        bids = ob['bids']
        asks = ob['asks']

        if bids and asks:  # Ensure there are bids and asks available
            best_bid = bids[0]['price']
            best_ask = asks[0]['price']
            spread = best_ask - best_bid

            tob_data.append({
                'date': date,
                'best_bid': best_bid,
                'best_ask': best_ask,
                'spread': spread
            })

    return tob_data

## <font color= #6B6B6B> Q5 : Display the timestamps above a particular spread value (2pt) </font>
<hr style="border:0.02in solid gray"> </hr>

In order to see some interesting time periods, lets see if we can find the timestamps where spread was above to 8.0 Usd.

In [ ]:
spread_threshold = 8.0
timestamps_above_threshold = df_ts_tob[df_ts_tob['spread'] > spread_threshold]['date']

## <font color= #6B6B6B> Q6 : Create a timeseries plot with bid and ask (Line plot) (2pt) </font>
<hr style="border:0.02in solid gray"> </hr>

Lets visualize how the bid and the ask have been in the whole time series data we have. You have to create a time series plot, using plotly, exactly like this one:

In [ ]:
def visualize_bid_ask(df_ts_tob):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_ts_tob['date'], y=df_ts_tob['best_bid'], name='Bid'))
    fig.add_trace(go.Scatter(x=df_ts_tob['date'], y=df_ts_tob['best_ask'], name='Ask'))

    fig.update_layout(
        title='Bid and Ask Prices over Time',
        xaxis_title='Date',
        yaxis_title='Price',
        hovermode='x'
    )

    fig.show()

# Example usage
visualize_bid_ask(df_ts_tob)

## <font color= #6B6B6B> Q7 : Create a boxplot graph with Spreads (5pt) </font>
<hr style="border:0.02in solid gray"> </hr>

We will need to see the spread in time batches, so, lets see if you can split the data into hours and create a boxplot for every group, like the following:

In [ ]:
df_ts_tob['spread'].describe()

Hint: You will need to have the timestamp converted to only minutes, and use these two parameters when calling the plotly function: 
- boxpoints='outliers'   # can also be outliers, or suspectedoutliers, or False´
- jitter=0.1             # add some jitter for a better separation between points

In [ ]:
# Extract the hour from the date column
df_ts_tob['hour'] = pd.to_datetime(df_ts_tob['date']).dt.hour

# Group the data by hour and calculate the spread for each group
grouped_data = df_ts_tob.groupby('hour')['spread'].apply(list)

# Create a boxplot using Plotly Express
fig = px.box(df_ts_tob, x='hour', y='spread', title='Spread Distribution by Hour')
fig.show()

## <font color= #6B6B6B> Q8 : Create Orderbook Imbalance Metric for all Orderbooks (2pt) </font>
<hr style="border:0.02in solid gray"> </hr>

For the following calculations, you can use your own way and commands to create the code for those, yet, there are some suggestions for using numpy arrays. Feel free to focus on the levels of engineering development as mentioned in class, assign this priority level: 1) It works, 2) Its efficient, 3) Its elegant.

Orderbook Imbalance

Using only the Top of the Book (TOB) information, which means, only the Bid and Ask volumes. Or, more than 1 level up to a  level as depth selection for each side

\begin{equation}
{I}^{TOB}_{t} = \frac{V^{b}_{t}}{V^{b}_{t} + V^{a}_{t}} \quad , \quad I^{LOB}_{t} = \frac{\sum_{b=1}^{B} V^{b}_{t}}{\sum_{b=1}^{B} V^{b}_{t} + \sum_{a=1}^{A} V^{a}_{t}}
\end{equation}

In [ ]:
df_ts_tob_complete['imbalance']=[np.sum(df_ts_tob_complete['bid_size'][:i+1])/ \
                                 (np.sum(df_ts_tob_complete['bid_size'][:i+1])+ \
                                 np.sum(df_ts_tob_complete['ask_size'][:i+1]))
                                 for i in range(len(df_ts_tob_complete))]

df_imbalance=df_ts_tob_complete.copy()
df_imbalance.drop(['bid_size', 'bid', 'ask', 'ask_size', 'spread', 'midprice'], axis=1)

## <font color= #6B6B6B> Q9 : Create Spread Metric for all Orderbooks (2pt) </font>
<hr style="border:0.02in solid gray"> </hr>

Spread

Also known as Quote Spread, the most commonly used spread definition and the one we use predominantly in our systems.

\begin{equation}
Qs_{t} = P^{a}_{t} - P^{b}_{t}
\end{equation}

In [ ]:
#df_ts_tob_complete['spread']=df_ts_tob_complete['ask']-df_ts_tob_complete['bid']

df_spread=df_ts_tob_complete.copy()
df_spread.drop(['bid_size', 'bid', 'ask', 'ask_size', 'midprice', 'imbalance'], axis=1)

## <font color= #6B6B6B> Q10 : Create Midprice Metric for all Orderbooks (2pt) </font>
<hr style="border:0.02in solid gray"> </hr>

Midprice 

Also known as Arithmetic Midprice, the most commonly used formula and the one that is present in our systems.

\begin{equation}
M_{t} = \frac{1}{2} (P^{a}_{t} + P^{b}_{t})
\end{equation}

In [ ]:
df_midprice=df_ts_tob_complete.copy()
df_midprice.drop(['bid_size', 'bid', 'ask', 'ask_size', 'spread', 'imbalance'], axis=1)

## <font color= #6B6B6B> Q11 : Create Weighted Midprice Metric for all Orderbooks (2pt) </font>
<hr style="border:0.02in solid gray"> </hr>

Weighted-Midprice

It uses only the TOB prices and their corresponding volumes. It represents the weighted average of the Bid () and the Ask (), weighted by their relative volumes  and .

\begin{equation}
M^{\omega}_{t} = \frac{V^{a}_{t}}{V^{b}_{t} + V^{a}_{t}} P^{b}_{t} + \frac{V^{b}_{t}}{V^{b}_{t} + V^{a}_{t}} P^{a}_{t}
\end{equation}

If there are a lot of Buyers (), then the Weighted-Midprice is pushed towards (since  will multiply  and divide ), the opposite effect for the case of having a lot of Sellers is also true. This indicates the Buy or Sell pressure in the market and thus the tendency of the price to move either towards the Bid or Ask side.

In [ ]:
df_ts_tob_complete['weighted_midprice']=((df_ts_tob_complete['ask_size'])/ \
                                         ((df_ts_tob_complete['bid_size'])+ \
                                         (df_ts_tob_complete['ask_size'])))* \
                                         (df_ts_tob_complete['bid'])+ \
                                         ((df_ts_tob_complete['bid_size'])/ \
                                         ((df_ts_tob_complete['bid_size'])+ \
                                         (df_ts_tob_complete['ask_size'])))* \
                                         (df_ts_tob_complete['ask'])

df_imbalance=df_ts_tob_complete.copy()
df_imbalance.drop(['bid_size', 'bid', 'ask', 'ask_size', 'spread', 'midprice', 'imbalance'], axis=1)

## <font color= #6B6B6B> Q12 : Volume-Weighted Mid Price (4 pts)</font>
<hr style="border:0.02in solid gray"> </hr>

VWAP 

This is an acronym for Volume Weighted Average Price and is the average price between Bids () and Asks (), weighted by the volumes in Bid () and Ask (), and doing this using  as the selected depth for each side.

\begin{equation}
\text{VWAP}^{(B, A)}_{t} = \frac{\sum_{b=1}^{B} [P^{b}_{t} * V^{b}_{t}]  + \sum_{a=1}^{A} [ P^{a}_{t} * V^{a}_{t}]} {\sum_{b=1}^{B} V^{b}_{t} + \sum_{a=1}^{A} V^{a}_{t}}
\end{equation}

In [ ]:
df_ts_tob_complete['volume_weighted_midprice']=[(np.sum((df_ts_tob_complete['bid'][:i+1])* \
                                                (df_ts_tob_complete['bid_size'][:i+1]))+ \
                                                np.sum((df_ts_tob_complete['ask'][:i+1])* \
                                                (df_ts_tob_complete['ask_size'][:i+1])))/ \
                                                (np.sum(df_ts_tob_complete['bid_size'][:i+1])+ \
                                                np.sum(df_ts_tob_complete['ask_size'][:i+1]))
                                                for i in range(len(df_ts_tob_complete))]

df_imbalance=df_ts_tob_complete.copy()
df_imbalance.drop(['bid_size', 'bid', 'ask', 'ask_size', 'spread', 'midprice', 'imbalance', 'weighted_midprice'], axis=1)

Honestly, I had some trouble making the coding in modules, instead of using a single notebook. But I think I understood the trading structures and bid and ask method. 

# <font color= #6B6B6B> <font size = 6> References </font>
<hr style="border:0.02in solid gray"> </hr>

[1] Munnoz, 2020. Python project template. https://github.com/iffranciscome/python-project. (2021).

<br>